In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import tensorflow as tf
tf.VERSION

'1.15.0'

In [7]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.layers.advanced_activations import LeakyReLU

import keras.backend as K

img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 100

def Generator():
  model = Sequential()
  model.add(Dense(128 * 7 * 7, activation="relu", input_dim = latent_dim))
  model.add(Reshape((7, 7, 128)))
  model.add(UpSampling2D())
  model.add(Conv2D(128, kernel_size=4, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))
  model.add(UpSampling2D())
  model.add(Conv2D(64, kernel_size=4, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))
  model.add(Conv2D(channels, kernel_size=4, padding="same"))
  model.add(Activation("tanh"))
  model.summary()

  noise = Input(shape=(latent_dim,))
  img = model(noise)

  return Model(inputs = noise, outputs = img)

def Critic():
  model = Sequential()
  model.add(Conv2D(16, kernel_size=3, strides=2, input_shape = img_shape, padding="same"))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
  model.add(ZeroPadding2D(padding=((0,1),(0,1))))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(1))
  model.summary()

  img = Input(shape = img_shape)
  validity = model(img)

        
  return Model(inputs = img, outputs = validity)

def wasserstein_loss(y_true, y_pred):
  return K.mean(y_true * y_pred)
        
# Following parameter and optimizer set as recommended in paper
n_critic = 5
clip_value = 0.01
optimizer = RMSprop(lr=0.00005)

# Build and compile the critic
C = Critic()
C.compile(loss = wasserstein_loss, optimizer=optimizer, metrics=['accuracy'])

# Build the generator
G = Generator()

# The generator takes noise as input and generates imgs
z = Input(shape=(latent_dim,))
img = G(z)

# For the combined model we will only train the generator
C.trainable = False

# The discriminator takes generated images as input and determines validity
valid = C(img)

# The combined model  (stacked generator and critic)
# Trains the generator to fool the discriminator
combined = Model(inputs = z, outputs = valid)
combined.compile(loss = wasserstein_loss, optimizer=optimizer, metrics=['accuracy'])


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 14, 14, 16)        160       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 7, 7, 32)          4640      
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 8, 8, 32)          0         
_________________________________________________________________
batch_normalization_11 (Batc (None, 8, 8, 32)          128       
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 8, 8, 32)         

In [8]:
from keras.datasets import mnist
import matplotlib.pyplot as plt
import os
import numpy as np

os.chdir('/content/drive/My Drive/GitHub Repositories/Generative Models')
baseDir = './Wasserstein GAN'
outputDir = os.path.join(baseDir, 'outputs')
if not os.path.exists(outputDir):
  os.makedirs(outputDir)
  print('Output Directory Created to save Results')

epochs=5000
batch_size=64
sample_interval=50

# Load the dataset
(X_train, _), (_, _) = mnist.load_data()

# Rescale -1 to 1
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)

# Adversarial ground truths
valid = -np.ones((batch_size, 1))
fake = np.ones((batch_size, 1))

def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = G.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig(outputDir+'/image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()
    plt.close()


for epoch in range(epochs):
  epoch=epoch+1
  for _ in range(n_critic):

      # ---------------------
      #  Train Discriminator
      # ---------------------

      # Select a random batch of images
      idx = np.random.randint(0, X_train.shape[0], batch_size)
      imgs = X_train[idx]
      
      # Sample noise as generator input
      noise = np.random.normal(0, 1, (batch_size, latent_dim))

      # Generate a batch of new images
      gen_imgs = G.predict(noise)

      # Train the critic
      d_loss_real = C.train_on_batch(imgs, valid)
      d_loss_fake = C.train_on_batch(gen_imgs, fake)
      d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)

      # Clip critic weights
      for l in C.layers:
          weights = l.get_weights()
          weights = [np.clip(w, -clip_value, clip_value) for w in weights]
          l.set_weights(weights)


  # ---------------------
  #  Train Generator
  # ---------------------

  g_loss = combined.train_on_batch(noise, valid)

  # Plot the progress
  print ("%d [D loss: %f] [G loss: %f]" % (epoch, 1 - d_loss[0], 1 - g_loss[0]))

  # If at save interval => save generated image samples
  if epoch % sample_interval == 0:
      save_imgs(epoch)






Output hidden; open in https://colab.research.google.com to view.

In [0]:
import imageio
import glob
images = []
for file_name in os.listdir(outputDir):
    if file_name.endswith('.png'):
        file_path = os.path.join(outputDir, file_name)
        images.append(imageio.imread(file_path))
imageio.mimsave(os.path.join(baseDir, 'WGAN-training-outputs.gif'), images, fps = 3)